# Collect all settled locations

In [145]:
import geopandas as gpd
import pandas as pd
import geopy
import osmnx as ox
import numpy as np
import rasterio as rio
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp').sample(1000)
location_csv = gpd.GeoDataFrame()
location_csv['name'] = iq_locations['PNameEn']
location_csv['region'] = iq_locations['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = iq_locations['Latitude']
location_csv['lon'] = iq_locations['Longitude']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = 0
location_csv['pop/cap'] = 0
location_csv.head()

,name,region,country,lat,lon,location_type,conflict_date,pop/cap
16520,Qutan,Kirkuk,Iraq,35.567556,44.232194,town,0,0
23488,Shafi,Salah al-Din,Iraq,34.310000,44.280000,town,0,0
17219,Madhoriya,Kirkuk,Iraq,35.339611,43.819528,town,0,0
3830,Al-Shebana Village,Qadissiya,Iraq,32.267778,44.585000,town,0,0
1244,Al-Khalefah,Basrah,Iraq,30.933056,47.281389,town,0,0


# Get population numbers

In [146]:
import os
from shapely.geometry import Point
listy = []
old_cols = None
for each_file in os.listdir('iom_dtm_reports/'):
    print(each_file)
    df = pd.read_csv('iom_dtm_reports/' + each_file, usecols = ['Latitude', 'Longitude', 'Families', 'Individuals'])
    points = [Point(x,y) for x, y in zip(df.Longitude, df.Latitude)]
    df = gpd.GeoDataFrame(df)
    df.crs = iq_locations.crs
    df['geometry'] = gpd.GeoSeries(points).buffer(0.01) # expand to make sure we get matches
    df_join = gpd.sjoin(iq_locations, df)


d89.csv
d88.csv
r87.csv
r86.csv
r84.csv
r90.csv
r91.csv
r85.csv
r88.csv
r89.csv
d86.csv
d87.csv
d91.csv
d85.csv
d84.csv
d90.csv


In [67]:
df_join.

,PCode,PNameEn,PNameAr,Longitude_left,Latitude_left,PCodeAlt1,PNameAlt1,A1Code,A1NameEn,A2Code,A2NameEn,geometry,index_right,Latitude_right,Longitude_right,Families,Individuals
27,IQ-P00028,Al Batra Wal Sakhriyah (Al Heremat),,43.882217,33.164126,110213.0,Al Batra wal Sakhriyah (Al Heremat),IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.882217 33.164126),296,33.160010,43.890010,92,552
68,IQ-P00069,Albu Hori,البوحوري,43.891238,33.156752,110135.0,Albu Hori,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.891238 33.156752),296,33.160010,43.890010,92,552
91,IQ-P00092,Al-Huraimat,الهريمات,43.882217,33.164126,110148.0,Al-Huraimat,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.882217 33.164126),296,33.160010,43.890010,92,552
36,IQ-P00037,Al Madina al Siyahiya,المدينة السياحية,43.571611,33.244907,110219.0,Al Madina Al Siyahiya,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.571611 33.244907),1747,33.243370,43.572220,2383,14298
95,IQ-P00096,Al-Madina al-Siyahiya Complex,,43.571611,33.244907,110228.0,Al-Madina Al-Siyahiya Complex,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.571611 33.244907),1747,33.243370,43.572220,2383,14298
87,IQ-P00088,Al-Ekhaa Compaund,مجمع الاخاء السكني,43.865124,33.162342,109945.0,Al-Ekhaa Compaund,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.865124 33.162342),1082,33.163540,43.864580,65,390
133,IQ-P00134,Hay al Shaheed,,43.863435,33.163630,110214.0,Hay Al Shaheed,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.863435 33.16363),1082,33.163540,43.864580,65,390
140,IQ-P00141,Hay al Zihoor,,43.863435,33.163630,110216.0,Hay Al Zihoor,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.863435 33.16363),1082,33.163540,43.864580,65,390
115,IQ-P00116,Banat Abu al-Hasan,,44.068855,33.339031,110058.0,Banat Abu Al-Hasan,IQ-G01,Anbar,IQ-D002,Falluja,POINT (44.068855 33.339031),1809,33.337318,44.073877,48,288
123,IQ-P00124,Habaniya,,43.594300,33.379800,105701.0,Habaniya,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.5943 33.3798),1568,33.374490,43.588980,20,120


_Perform spatial join, then average out values across points_

In [147]:


denom = df_join.groupby('index_right').count().reset_index()
denom_dict = {}
for i in denom.itertuples():
    denom_dict[i.index_right] = i.PCode
    
adj_families = []
adj_individuals = []
for i in df_join.itertuples():
    adj_families.append(int(round(i.Families/denom_dict[i.index_right])))
    adj_individuals.append(int(round(i.Individuals/denom_dict[i.index_right])))
    
    
df_join['adj_families'] = adj_families
df_join['adj_individuals'] = adj_individuals


#pop87 = pd.read_csv('data/undata_1987_census/UNdata_Export_20180505_224541956.csv')


In [148]:
location_csv = gpd.GeoDataFrame()
location_csv['name'] = df_join['PNameEn']
location_csv['region'] = df_join['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = df_join['Latitude_left']
location_csv['lon'] = df_join['Longitude_left']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = 0
location_csv['HH_pop'] = df_join['adj_families']
location_csv['individual_pop'] = df_join['adj_individuals']

In [175]:
# now do conflict locations

from shapely.geometry import Point
conflict_loc = pd.read_csv('data/acled_unprocessed_conflict_locations.csv')
conflict_loc_geo = gpd.GeoDataFrame(conflict_loc)
points = [Point(float(j), float(i)) for i, j in zip(conflict_loc.latitude, conflict_loc.longitude)]
geometries_acled = gpd.GeoSeries(points).buffer(0.1) # buffer is somewhat arbitrary...
# align geometries - can't know what geometry acled was using...
acled_locations = gpd.GeoDataFrame()
acled_locations['geometry'] = geometries_acled
acled_locations.crs = iq_locations.crs
acled_locations.to_crs(iq_locations.crs)
acled_locations['date'] = conflict_loc['event_date']
acled_locations.to_file('data/acled_conflict_geometries.shp')


In [ ]:
location_csv_tmp = iq_locations
#location_csv_tmp.crs = acled_locations.crs
location_csv_tmp = gpd.sjoin(location_csv_tmp, acled_locations)

In [272]:
loc_list = location_csv_tmp.groupby('date').apply(lambda x: set(x.PNameEn))
conf_locs = pd.DataFrame(loc_list)
conf_locs['day'] = np.arange(len(conf_locs))



In [273]:
conf_locs.to_csv('conflict_locations_test.csv')

In [274]:
conf_locs = conf_locs.reset_index()

In [275]:
date_int = [int("".join([j for j in i if j != '-'])) for i in conf_locs.date]

In [291]:

def assign_round(x):
    start = 20170101
    # from http://iraqdtm.iom.int/IDPsML.aspx
    upper_dates = [20170105, 20170119, 20170202, 20170216, 
                  20170302, 20170316, 20170330, 20170413, 
                  20170427, 20170515, 20170530, 20170615,
                  20170630, 20170715, 20170730, 20170815,
                  20170830, 20170915, 20170930, 20171015, 
                  20171031, 20171115, 20171129, 20171215,
                  20171231,
                  20180115,20180131, 20180215, 20180228, 
                  20180315, 20180330, 20180415, 20180430,
                  ]
    top_val = len(upper_dates)
    offset = len(upper_dates)
    
    for i in upper_dates:
        if x > i:
            offset -= 1
            start = i
        else:
            return top_val - offset
        
conf_locs['round'] = [assign_round(i) for i in date_int]

    

In [292]:
conf_locs

,date,0,day,round
0,0,"{Hay Aljza'Ir 2, Al-Noor, Alchuwaiha Min Alswa...",0,0
1,0,"{Maita, Hay al Rea'Sah, Babil, Al-Gadeer, Tak ...",1,0
2,0,"{Hay al Rea'Sah, Babil, Al Basateen - Mahala 3...",2,0
3,0,"{Hay al Rea'Sah, Babil, Hay al Shurta - 865, A...",3,0
4,0,"{Hay al Rea'Sah, Babil, Al Basateen - Mahala 3...",4,0
5,1,"{Hay al Rea'Sah, Babil, Al Basateen - Mahala 3...",5,1
6,1,"{Al-Qahira, Ali Rish Village, Olmush, Al Sabti...",6,1
7,1,"{Hay al Rea'Sah, Babil, Tak Tak, Al Basateen -...",7,1
8,1,"{Door Nawab al-Dhabat Fei al-Wiseiah, Hay al R...",8,1
9,1,"{Maita, Hay al Rea'Sah, Babil, Khwaishija, Al ...",9,1


# Above was taking too long, considering option of matches for locations and worldpop raster

In [229]:
'''# 2015 projections from WorldPop, adjusted to UN estimates
pops_raster = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')
'''

"# 2015 projections from WorldPop, adjusted to UN estimates\npops_raster = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')\n"

In [514]:
'''#http://www.perrygeo.com/python-affine-transforms.html
pop_pred = []
pops_dat = pops.read().squeeze()
translator = pops.affines
with open('pops_pred.txt', 'w') as f:
    for y in range(pops_dat.shape[0]):
        for x in range(pops_dat.shape[1]):
            long, lat = translator * (x,y) # remember to swap b/c this is x,y
            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))
# takes too long, try another way'''

'#http://www.perrygeo.com/python-affine-transforms.html\npop_pred = []\npops_dat = pops.read().squeeze()\ntranslator = pops.affines\nwith open(\'pops_pred.txt\', \'w\') as f:\n    for y in range(pops_dat.shape[0]):\n        for x in range(pops_dat.shape[1]):\n            long, lat = translator * (x,y) # remember to swap b/c this is x,y\n            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))\n# takes too long, try another way'

In [521]:
'''translator = pops_raster.affine
pops_dat = pops_raster.read().squeeze()
iq_locs_pop = location_csv.copy()
pops = []
import numpy as np
for i in iq_locs_pop.itertuples():
    x = i.lon
    y = i.lat
    col, row = ~ translator * (x, y)
    col = int(round(col,0))
    row = int(round(row,0))
    pops.append(int(pops_dat[row, col] * 10))
    
location_csv['pop_density'] = np.arange(0, len(location_csv))
location_csv.to_csv('data/iq_locs_pop.csv')'''

# now getting the routes between all locations
start with just doing great circle distance, then maybe move to routing values

In [151]:
dist_iq_locations = iq_locations.copy()

In [ ]:
'''dist_df = pd.DataFrame()
for i in dist_iq_locations.itertuples():
    dist_df[i.PNameEn] = dist_iq_locations.geometry.distance(i.geometry)
dist_df.index = dist_df.columns
dist_df.to_csv('distance_matrix.csv')'''

In [152]:
trips = set()
count = 0
with open("flee-release/routes.csv", "w") as f:
    #f.write(",".join(["name1", "name2", "distance\n"]))
    for i in dist_iq_locations.itertuples():
        dists = dist_iq_locations.geometry.distance(i.geometry) * 1000
        dists = [int(round(i,0)) for i in dists]
        tups = zip(dist_iq_locations.PNameEn, dists)
        for destination, time in tups:
            key = (i.PNameEn, destination)
            if key[0] == key[1]:
                continue
            if key not in trips:
                trips.add(key)
                trips.add((key[1], key[0]))
                f.write(",".join([key[0], key[1], str(time)]) + "\n")
                if key[0] != key[1]:
                    f.write(",".join([key[1], key[0], str(time)]) + "\n")

# conflict locations for location_csv

In [ ]:
'''conflict_loc = pd.read_csv('https://data.humdata.org/hxlproxy/data.csv?url=https%3A%2F%2Fapi.acleddata.com%2Facled%2Fread.csv%3Flimit%3D0%26iso%3D368&name=ACLEDHXL&tagger-match-all=on&tagger-02-header=iso&tagger-02-tag=%23country%2Bcode&tagger-03-header=event_id_cnty&tagger-03-tag=%23event%2Bcode&tagger-05-header=event_date&tagger-05-tag=%23date%2Boccurred+&tagger-08-header=event_type&tagger-08-tag=%23event%2Btype&tagger-09-header=actor1&tagger-09-tag=%23group%2Bname%2Bfirst&tagger-10-header=assoc_actor_1&tagger-10-tag=%23group%2Bname%2Bfirst%2Bassoc&tagger-12-header=actor2&tagger-12-tag=%23group%2Bname%2Bsecond&tagger-13-header=assoc_actor_2&tagger-13-tag=%23group%2Bname%2Bsecond%2Bassoc&tagger-16-header=region&tagger-16-tag=%23region%2Bname&tagger-17-header=country&tagger-17-tag=%23country%2Bname&tagger-18-header=admin1&tagger-18-tag=%23adm1%2Bname&tagger-19-header=admin2&tagger-19-tag=%23adm2%2Bname&tagger-20-header=admin3&tagger-20-tag=%23adm3%2Bname&tagger-21-header=location&tagger-21-tag=%23loc%2Bname&tagger-22-header=latitude&tagger-22-tag=%23geo%2Blat&tagger-23-header=longitude&tagger-23-tag=%23geo%2Blon&tagger-25-header=source&tagger-25-tag=%23meta%2Bsource&tagger-27-header=notes&tagger-27-tag=%23description&tagger-28-header=fatalities&tagger-28-tag=%23affected%2Bkilled&header-row=1')
conflict_loc = conflict_loc.iloc[1:]
conflict_loc.to_csv("data/acled_unprocessed_conflict_locations.csv")'''


In [525]:
from shapely.geometry import Point
conflict_loc = pd.read_csv('data/acled_unprocessed_conflict_locations.csv')
conflict_loc_geo = gpd.GeoDataFrame(conflict_loc)
points = [Point(float(j), float(i)) for i, j in zip(conflict_loc.latitude, conflict_loc.longitude)]
geometries_acled = gpd.GeoSeries(points).buffer(0.1) # buffer is somewhat arbitrary...
# align geometries - can't know what geometry acled was using...
acled_locations = gpd.GeoDataFrame()
acled_locations['geometry'] = geometries_acled
acled_locations.crs = iq_locations.crs
acled_locations.to_crs(iq_locations.crs)
acled_locations['date'] = conflict_loc['event_date']
acled_locations.to_file('data/acled_conflict_geometries.shp')

In [ ]:
acled_temp = gpd.sjoin(iq_locations, acled_locations, 'left')
acled_temp.to_file('data/acled_joined_locations.shp')

# for now randomly assign conflict

In [527]:
#acled_temp.sample(1000).to_file('data/acled_joined_locations_sample.shp')
#pop_list  = location_csv['pop_density'] * 10 # some pop
#location_csv['pop/cap'] = [round(min(i, 0)) for i in pop_list]
pops_list = [i * 10 for i in location_csv.pop_density]
print(max(location_csv.pop_density))
location_csv['pop/cap'] = pops_list
#location_csv_new['pop/cap'] = round(location_csv_new['pop/cap'])
#location_csv = location_csv.iloc[:,:-1]
import numpy as np
conflict_list = []
conflict_type = []
for i in range(len(location_csv)):
    one = np.random.randint(0,2)
    if one > 0:
        conflict_type.append('conflict')
        one = np.random.randint(1,50)
    else:
        if np.random.randint(0,2) > 0:
            conflict_type.append('camp')
        else:
            conflict_type.append('town')
    conflict_list.append(one)
location_csv['conflict_date'] = conflict_list
location_csv['location_type'] = conflict_type
location_csv_new = location_csv.iloc[:,:-1]
location_csv_new.to_csv('flee-release/testing_locations_data.csv', index = False, header=False)


499


In [ ]:
#locations_sample = gpd.read_file('data/acled_joined_locations_sample.shp')
#conflict_locations = gpd.read_file('data/acled_joined_locations.shp')
#conflict_locations.head()
#location_csv_new['pop/cap'] = [int(i) for i in location_csv_new['pop/cap']]

In [ ]:
locations_sample = acled_temp.copy()
just_date = []
for i in locations_sample.itertuples():
    try:
        if '-' in i.date:
            just_date.append(int("".join([i for i in i.date if i != '-'])))
        else:
            just_date.append(-1)
    except:
        just_date.append(-1)
            
locations_sample['justdate'] = just_date
locations_sample.sort_values('justdate', inplace = True)
date_rank = []
datessofar = set()
current = -1
for i in locations_sample.itertuples():
    if i.justdate != -1:
        if i.justdate not in datessofar:
            current += 1
            datessofar.add(i.justdate)
        date_rank.append(current)
    else:
        date_rank.append(None)
locations_sample['conflict_start'] = date_rank
    
        
    

In [ ]:
conflict_list = []
conflicts = set()
for i in locations_sample.itertuples():
    if i.justdate >= 0:
        if str(i.Longitude) + str(i.Latitude) not in conflicts:
            conflicts.add(str(i.Longitude) + str(i.Latitude))
            conflict_list.append(i)
conflict_df = gpd.GeoDataFrame(conflict_list)
conflict_df

In [ ]:
'''location_csv_tmp = location_csv.copy()
points = [Point(i.lon, i.lat) for i in location_csv_tmp.itertuples()]
location_csv_tmp['geometry'] = points
conflict_list = []
type_list = []
for i in location_csv_tmp.itertuples():
    for j in conflict_df.itertuples():
        if i.geometry == j.geometry:
            conflict_list.append([i.geometry, j.conflict_start])
            break
    '''

#location_csv_tmp_joined = gpd.sjoin(location_csv_tmp, conflict_df, 'left')
#location_csv_tmp_joined

In [ ]:
df = pd.read_csv('flee-release/simulation_results.csv').T.reset_index()
df.columns = ['location'] +  list(df.columns[1:])
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp')
res_df = iq_locations.merge(df, left_on = 'PNameEn', right_on = 'location')
for i in range(10):
    res_df.plot(i)